## - 개요

화장품 전성분 데이터를 수집하기 위한 부작업.
<br>
이전 수집한 kca_ing_name.csv 파일에 들어있는 전성분 데이터를 활용,
<br>
INCI-Decoder 사이트 내 성분페이지에 접근하여
<br>
각 성분이 포함된 모든 제품의 리스트를 가져오기.
<br>
<br>
<br>
(이때, 중복은 허용하지 않으며 활용하는 모든 csv 파일은
<br>
실행 파일과 같은 디렉토리에 있는 것을 원칙으로 함)

### 1. 활용 모듈 및 성분 파일 불러오기

프로젝트에서 활용할 모듈은 사용한 셀에서 설명할 예정

In [12]:
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
import tqdm

기존에 KCA 성분사전에서 수집했던 성분 데이터 불러오기.
<br>
해당 파일의 'formatted_영문명' 칼럼을 활용하여
<br>
INCI-Decoder 사이트 내 제품명 데이터에 접근할 것.

In [13]:
# 성분 파일(csv) 불러오기

ing_df = pd.read_csv('kca_ing_name.csv',index_col=0)
ing_df['formatted_영문명'] = ing_df['formatted_영문명'].str.replace("/","-")

In [14]:
# 데이터 프리뷰 (top 5)

ing_df.head()

,성분코드,성분명,영문명,formatted_영문명
0,1,가공소금,NaN,NaN
1,2,가지열매추출물,Solanum Melongena Fruit Extract,solanum-melongena-fruit-extract
2,3,구멍쇠미역추출물,Agarum Cribrosum Extract,agarum-cribrosum-extract
3,4,루핀아미노산,Lupine Amino Acids,lupine-amino-acids
4,5,류신,Leucine,leucine


In [15]:
# 데이터 프리뷰 (bottom 5)

ing_df.tail()

,성분코드,성분명,영문명,formatted_영문명
24651,23301,락토바실러스/(참당귀뿌리/병풀/더덕뿌리/감초뿌리/홍삼뿌리/쇠비름)추출물발효여과물,NaN,NaN
24652,23302,생강나무가지/잎수,NaN,NaN
24653,23302,생강나무가지/잎수,NaN,NaN
24654,23303,생강나무가지/잎추출물,NaN,NaN
24655,23303,생강나무가지/잎추출물,NaN,NaN


### 2. 데이터 수집을 위한 사전 처리

크롤링에 활용할 데이터만 남기는 과정.
<br>
수집에 앞서 웹페이지의 특성을 파악하고,
<br>
(각 페이지 정보를 순차적으로 불러오고, 특정 태그 내 제품 이름 정보만 수집)
<br>
페이지의 특성에 맞는 방식으로 데이터를 수집하기 위해
<br>
세 가지 함수 설정.

In [18]:
# 성분으로 조회한 제품 리스트 만들기

def add_ing_products(tags,product_names):  # tags = html tag
    for tag in tags:
        product = tag.attrs['data-ga-eventlabel'][8:]
        product_names.add(product)
            
    return product_names

In [19]:
# 성분으로 접근한 웹페이지의 제품 리스트에 '다음페이지'가 존재하는지 확인

def next_page_exists(soup):
    if "Next" in soup.find(id="product").find_all("div")[-1].text: 
        return True
    else:
        return False

In [20]:
# 제품 리스트 만들기

def get_product_list(ing,product_names):
    url = 'https://incidecoder.com/ingredients/'+ ing # 표준화된 성분명에 따라서 url 형성
    
    # url이 존재한다면(성분에 해당하는 제품이 있다면)
    try:
        html = urlopen(url) 
        source = html.read()
        soup = BeautifulSoup(source, "html.parser")
        tags = soup.select("#product > div > a") 
        add_ing_products(tags,product_names) 

        if next_page_exists(soup): # url 내 다음 페이지가 존재한다면 작용
            nextpage = True
        
        while nextpage: # 다음 페이지가 있다면 반복
            nexturl = soup.find(id="product").find_all("a")[-1]['href'] 
            url = 'https://incidecoder.com'+ nexturl 
            html = urlopen(url) 
            source = html.read()
            soup = BeautifulSoup(source, "html.parser")
            tags = soup.select("#product > div > a")
            add_ing_products(tags,product_names) 

            if not next_page_exists(soup): # url 내 다음 페이지가 존재하지 않으면 정지
                nextpage = False
    
    # url이 존재하지 않는다면(성분에 해당하는 제품이 없다면)
    except Exception:
        pass
    
    return product_names

### 3. 데이터 수집

각 성분에 해당하는 페이지를 순차적으로 돌면서 
<br>
해당 성분이 포함된 모든 제품의 품명을 수집하는 과정.
<br>
'product_label' 이라는 단일 칼럼으로 구성된 데이터프레임을
<br>
csv 파일로 바꾸는 과정까지 포함
<br>
<br>
<br>
(이 과정은 보통의 노트북에서 단일 실행 시 30시간이 넘게 걸리므로,
<br>
실행할 시 작업을 적절한 수준으로 분할하여 실행할 것을 권장)

In [25]:
# 이전 과정에서의 성분 테이블에서 '영문명' 칼럼에 해당하는 데이터만 불러오기
kca_ing_eng = list(ing_df['formatted_영문명'].dropna())

# 불러온 리스트의 길이 확인
print('성분의 수 (KCA 성분사전 - ENG only):', len(kca_ing_eng))

성분의 수 (KCA 성분사전 - ENG only): 23158


2개의 성분들은 데이터 수집 과정에서 제외 (Water, Glycerin).
<br>
대부분의 제품들에 들어 있는 성분으로 수집 과정이 과도하게 길어지는 것 방지 
<br>
(예시: 'Water'는 1342페이지의 제품 리스트가 존재)

In [27]:
# 2개 성분 제외 : 물과 글리세린

remove_items = ['water','glycerin']

for item in remove_items:
    try: 
        kca_ing_eng.remove(item)
    except ValueError:
        pass

print('성분의 수 (KCA 성분사전 - ENG only):', len(kca_ing_eng))

성분의 수 (KCA 성분사전 - ENG only): 23156


In [29]:
# 테스트 코드

tmp_products = set()
for ing in tqdm.notebook.tqdm(kca_ing_eng[:5]):
    tmp_products.update(get_product_list(ing,tmp_products))

tmp_products_df = pd.DataFrame(columns=['product_name'],data=list(tmp_products))
tmp_products_df.head()

  0%|          | 0/5 [00:00<?, ?it/s]

,product_name
0,isntree-onion-newpair-cleansing-foam
1,ahc-hydrating-essential-real-eye-cream-for-face
2,filorga-nctf-intensive-serum
3,truly-blueberry-kush-cbd-body-oil
4,dr-jart-sleeping-mask


In [ ]:
# 기본 코드

# products = set()

# for ing in notebook.tqdm(kca_ing_eng):
#     products.update(get_product_list(ing,products))

# product_df = pd.DataFrame(columns=['product_name'],data=list(products))
# product_df.head()

실제 수집 과정에서, 'kca_ing_eng' 리스트는 수집 과정에서
<br>
다중 리소스 활용을 위해 구획별로 나누어 기본 코드를 통해 실행되었음.

In [34]:
# 실제 수집 과정에서의 구획 나누기

total = len(kca_ing_eng)
length = len(kca_ing_eng)//7
start = 0
end = length
i = 0

while True:
    sample_name = 'product_df'+str(i)
    print(f'{sample_name}: kca_ing_eng[{start}:{end}]')
    
    start = end 
    end += length
    total -= length
    i += 1
    
    if total < 0 or end > len(kca_ing_eng):
        sample_name = 'product_df'+str(i)
        print(f'{sample_name}: kca_ing_eng[{start}:]')
        break

product_df0: kca_ing_eng[0:3308]
product_df1: kca_ing_eng[3308:6616]
product_df2: kca_ing_eng[6616:9924]
product_df3: kca_ing_eng[9924:13232]
product_df4: kca_ing_eng[13232:16540]
product_df5: kca_ing_eng[16540:19848]
product_df6: kca_ing_eng[19848:23156]
product_df7: kca_ing_eng[23156:]


이후 나누어진 구획에 따라서 각자 크롤링 과정을 거침.
<br>
아래의 코드 블럭은 7번 구획에 대한 수집 방식에 대해서 예시를 든 것.

In [ ]:
# product_df7: kca_ing_eng[19411:]

products = set()

for ing in tqdm.notebook.tqdm(kca_ing_eng[19411:]):
    products.update(get_product_list(ing,products))

products_df7 = pd.DataFrame(columns=['product_name'],data=list(products))
products_df7.head()
# product_df7.to_csv(product_df7.csv, index=False)

최종적으로 수집된 구획별 제품명 csv 파일을
<br>
병합하여 하나의 제품명 리스트를 구성함.

In [ ]:
# 병합 과정

# product_df0 = pd.read_csv('./product_df0.csv',index_col=False)
# product_df1 = pd.read_csv('./product_df1.csv',index_col=False)
# product_df2 = pd.read_csv('./product_df2.csv',index_col=False)
# product_df3 = pd.read_csv('./product_df3.csv',index_col=False)
# product_df4 = pd.read_csv('./product_df4.csv',index_col=False)
# product_df5 = pd.read_csv('./product_df5.csv',index_col=False)
# product_df6 = pd.read_csv('./product_df6.csv',index_col=False)
# product_df7 = pd.read_csv('./product_df7.csv',index_col=False)

# product_all = pd.concat([product_df0,product_df1,product_df2,product_df3,product_df4,product_df5,product_df6,product_df7],ignore_index=True).drop_duplicates()
# product_all.to_csv('product_df.csv',index=False)

### 4. 수집 데이터 확인

대량으로 수집한 데이터를 확인하는 과정.
<br>
수집된 데이터가 이상이 없는지 확인.

In [30]:
product_df = pd.read_csv('product_df.csv',index_col=False)
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77001 entries, 0 to 77000
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_label  77001 non-null  object
dtypes: object(1)
memory usage: 601.7+ KB


In [31]:
product_df

,product_label
0,clicks-expert-shine-control-foaming-cleanser
1,jvn-shine-drops
2,innisfree-auto-eyebrow-pencil-no-4
3,lululun-over45-camelia-pink
4,bielenda-good-skin-acid-peel-micro-exfoliating...
...,...
76996,lumene-hydration-recovery-aerating-gel-mask-2
76997,nivea-sensitive-post-shave-balm-2
76998,revivserums-ultimate-serum-2
76999,innisfree-super-volcanic-pore-clay-mask-2x
